**Michael Bocamazo**

**Question 3**: How well does a 2-stage model perform with logistic regression and minimal feature engineering?

**Date**: 2016/11/27

**Methods**: Select down to a few features, train multistage model with split on car availability, and cross-validate to find ridge regression term.  Examine resultant weights in both models.

**Conclusion**: 

In [1]:
import os, sys
import csv
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import seaborn as sns
import copy
%matplotlib inline
sns.set_style("darkgrid", {"grid.linewidth": .5, "axes.facecolor": ".9"})

In [3]:
import ML_utils as ml_ut

From the previous question, we have a clean data frame, and a data frame with expanded features.

In [4]:
df_c = pd.read_csv('SM_clean.csv')

In [5]:
df_c.columns

Index(['SURVEY', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE',
       'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'CAR_AV', 'TRAIN_TT',
       'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS', 'CAR_TT',
       'CAR_CO', 'CHOICE'],
      dtype='object')

In [6]:
df_e = pd.read_csv('SM_expand.csv')

In [7]:
df_e.columns

Index(['SURVEY', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE',
       'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'CAR_AV', 'TRAIN_TT',
       'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS', 'CAR_TT',
       'CAR_CO', 'CHOICE', 'min_CO', 'ratio_TRAIN_CO', 'ratio_SM_CO',
       'ratio_CAR_CO', 'min_TT', 'ratio_TRAIN_TT', 'ratio_SM_TT',
       'ratio_CAR_TT'],
      dtype='object')

### Features to include
We can make a simple list of features that would be appropriate for a logistic regression model without any transformations.  The certain ones are the numerics and binary features.  Less certain are the ordinal encoded features.  The simply encoded features require an OHE.

The **pure feats** are the numeric features that we expect to get a straight correlation to choice, except for cost, which needs an adjustment based on GA to work well.  

The **simple feats** are binary or ordinal encoded features.  The INCOME and AGE features both have a category for unknowns, at the endpoint.  If the latent classes behind the unknowns are equally distributed among the values, this acts as a regularizer.  In both of these features they occupy the greatest value.  They could be made to occupy the the mid value for a cleaner regularization.

The **ratio features** are those developed in the first pass to compare between alternatives.  These should be quite useful for random forests, because they condense the number of nodes needed to express the comparison of the cost features.  However, they might not be useful for logistic regression.  We can experiment.

Finally, there are 5 non-ordinal **encoded features** that we'll probably omit until we want much higher complexity.

In [8]:
simple_feats = ['SURVEY','LUGGAGE','INCOME','AGE','GA','SM_SEATS','CAR_AV','FIRST','MALE']
pure_feats = ['TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'CAR_TT','CAR_CO']
encode_feats = ['PURPOSE','TICKET','WHO','ORIGIN','DEST']
ratio_feats = ['min_CO', 'ratio_TRAIN_CO','ratio_SM_CO', 'ratio_CAR_CO', 'min_TT', 'ratio_TRAIN_TT',
               'ratio_SM_TT','ratio_CAR_TT']

## Multivariate Logistic Regression
Or, "Multinomial Logit"

In [9]:
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegressionCV

In [10]:
X = df_e.drop("CHOICE", axis = 1)
y = df_e['CHOICE']

The split was chosen based on session-split in q_00.

In [11]:
split = 7002

In [12]:
Xtrain, Xtest, ytrain, ytest = ml_ut.tt_split(X,y,split)

To start, we'll take the pure numeric feats and the simple ordinal encodings or binary features

In [13]:
ML_feat = pure_feats + simple_feats

In [14]:
Xtrain_car_av = Xtrain[Xtrain['CAR_AV']==1]

In [15]:
Xtrain_car_av_index = Xtrain['CAR_AV']==1

In [16]:
len(Xtrain)

7002

In [17]:
len(Xtrain_car_av)

5319

Most of the respondents in the chosen training set have a car available.  But it is important for the ~20% that do not that they have a different model, and are not mixed in.

In [18]:
ytrain_car_av = ytrain[Xtrain_car_av_index]

In [19]:
ytrain_car_av.value_counts()

2    3414
3    1271
1     634
Name: CHOICE, dtype: int64

CHOICE 1,2,3 = Train, SM, Car

In [20]:
ytrain.value_counts()

2    4453
1    1278
3    1271
Name: CHOICE, dtype: int64

This confirms that there are no car choices where it is not available.

In [21]:
ytrain_car_non = ytrain_car_av == 3

### Pipelining structure

In [22]:
import sklearn.pipeline

In [23]:
scaler = sklearn.preprocessing.StandardScaler()
clf = LogisticRegression()
model = sklearn.pipeline.Pipeline([('scaler',scaler),('LogReg',clf)])

In [24]:
model.fit(Xtrain_car_av[ML_feat], ytrain_car_non)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LogReg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [25]:
dy_x = model.predict(Xtrain_car_av[ML_feat])

For the sake of the argument, let's look at the training error.

In [26]:
sklearn.metrics.accuracy_score(ytrain_car_non, dy_x)

0.79375822523030648

The equivalent test set would be those in which a car is available; predict car-non

In [27]:
Xtest_car_av = Xtest[Xtest['CAR_AV']==1]
ytest_car_non = ytest[Xtest['CAR_AV']==1]==3 # this indexes into the CAR_AV examples
# and checks if the chosen mode is CAR

In [28]:
dy_x_test = model.predict(Xtest_car_av[ML_feat])

In [29]:
sklearn.metrics.accuracy_score(ytest_car_non, dy_x_test)

0.62389023405972555

In [30]:
sklearn.metrics.confusion_matrix(ytest_car_non, dy_x_test)

array([[1702,  206],
       [1192,  617]])

Vertical is true label, horizontal is predicted label.  The prediction skews too heavily towards not car choice.

In [31]:
ytest_car_non.value_counts()/len(ytest_car_non)

False    0.513317
True     0.486683
Name: CHOICE, dtype: float64

This means that the model has some discriminative power.  However, we are probably looking for something much better - this is only the first part of the model!

We cannot include the breakout into multiple subsets in the pipeline because they are not proper 'transforms' - they do not preserve the shape of the data.  So we need to write it in a different functional style.

In [32]:
Xtrain_car_av = Xtrain[Xtrain['CAR_AV']==1]

In [33]:
Xtrain_car_av_index = Xtrain['CAR_AV']==1

In [34]:
len(Xtrain_car_av_index)

7002

In [35]:
Xtrain_pTR = Xtrain_car_av[~dy_x]

ytrain_pTR = ytrain_car_av[~dy_x]

Xtrain_no_car = Xtrain[~Xtrain_car_av_index]

ytrain_no_car = ytrain[~Xtrain_car_av_index]

Xtr_stage2 = pd.concat([Xtrain_pTR, Xtrain_no_car])
ytr_stage2 = pd.concat([ytrain_pTR, ytrain_no_car])

The unchanged label vector will include car choices - should this be retained?

In [36]:
scaler2 = sklearn.preprocessing.StandardScaler()
clf2 = LogisticRegression()
model2 = sklearn.pipeline.Pipeline([('scaler',scaler2),('LogReg',clf2)])

In [37]:
model2.fit(Xtr_stage2[ML_feat], ytr_stage2)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LogReg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [39]:
dy_x2 = model2.predict(Xtr_stage2[ML_feat])

In [40]:
pd.DataFrame(dy_x2)[0].value_counts()

2    5783
1     626
3      47
Name: 0, dtype: int64

In [41]:
sum(ytrain_car_av[dy_x] == 3)/len(ytrain_car_av[dy_x])

0.65934065934065933

In [55]:
(sklearn.metrics.accuracy_score(ytr_stage2, dy_x2)*len(ytr_stage2) + sum(ytrain_car_av[dy_x]==3))/len(ytrain)

0.69665809768637532

This is our accuracy score.  But it does not give insight into the failure modes.  Also, it is worse than the previous model in training error.  Want to make the actual label vector for further analysis.

Then do the same breakout for test

In [56]:
Xtest_car_av_index = Xtest['CAR_AV']==1
ytest_car_av = ytest[Xtest_car_av_index]

Xtest_pTR = Xtest_car_av[~dy_x_test]
ytest_pTR = ytest_car_av[~dy_x_test]

Xtest_no_car = Xtest[Xtest['CAR_AV']!=1]
ytest_no_car = ytest[Xtest['CAR_AV']!=1]

Xte_stage2 = pd.concat([Xtest_pTR, Xtest_no_car])

yte_stage2 = pd.concat([ytest_pTR, ytest_no_car])

dy_x2_test = model2.predict(Xte_stage2[ML_feat])

In [57]:
pd.DataFrame(dy_x2_test)[0].value_counts()

2    2747
3     147
Name: 0, dtype: int64

No SwissMetro predictions included, and some cars!  This sequence didn't work very well.

In [58]:
sum(ytest_car_av[dy_x_test]==3)/len(ytest_car_av[dy_x_test])

0.74969623329283108

Better than the training set??

In [59]:
(sklearn.metrics.accuracy_score(yte_stage2, dy_x2_test)*len(yte_stage2)+sum(ytest_car_av[dy_x_test]==3))/len(ytest)

0.60532687651331718

The second model performed much worse.  I wonder if the transformations, that is, the scalings, are done correctly.

In [62]:
len(dy_x)

5319

In [68]:
sum(dy_x) + len(dy_x2)

7002

In [65]:
sum(dy_x)/len(dy_x)

0.10265087422447829

In [101]:
def construct_label_vec(X, dy_x1, dy_x2):
    pointer_1 = 0
    pointer_2 = 0
    pointer_label = 0
    label_vec = np.zeros((len(X),1))
    for i in X.index:
        if X.loc[i,'CAR_AV'] == 1:
            if dy_x1[pointer_1]==1:
                label_vec[pointer_label] = 3 # this is the code for CAR
            else:
                label_vec[pointer_label] = dy_x2[pointer_2]
                pointer_2 += 1
            pointer_1 += 1
        else:
            label_vec[pointer_label] = dy_x2[pointer_2]
            pointer_2 += 1
        pointer_label +=1 
    print(pointer_1)
    print(pointer_2)
    return label_vec
            

In [89]:
a = construct_label_vec(Xtrain, dy_x, dy_x2)

5319
6456


In [86]:
sklearn.metrics.accuracy_score(ytrain, a)

0.60554127392173662

In [104]:
sklearn.metrics.confusion_matrix(ytrain, a)

array([[  79, 1179,   20],
       [ 445, 3798,  210],
       [ 102,  806,  363]])

In [102]:
b = construct_label_vec(Xtest, dy_x_test, dy_x2_test)

3717
2894


In [103]:
sklearn.metrics.accuracy_score(ytest, b)

0.60532687651331718

In [105]:
sklearn.metrics.confusion_matrix(ytest, b)

array([[   0,  114,   31],
       [   0, 1537,  226],
       [   0, 1096,  713]])

array([[ 2.],
       [ 2.],
       [ 2.],
       ..., 
       [ 2.],
       [ 2.],
       [ 2.]])

if the CAR_AV is true, don't advance the 2nd pointer

for every sample in the set
if the CAR_AV is true, look at dy_x first
    if dy_x is true, label it as true, then advance dy_x
    if dy_x is false, label is according to dy_x2, then advance dy_x and dy_x2
if CAR_AV is false,
    label it according to dy_x2, and advance dy_x2
    
dy_x advances according to number of CAR_AV
dy_x2 advances according to CAR_AV && ~dy_x + ~CAR_AV
    

In [69]:
len(dy_x)

5319

In [73]:
sum(Xtrain['CAR_AV']==1)

5319

In [77]:
sum(Xtrain['CAR_AV']!=1)

1683

In [80]:
sum(~dy_x)

4773

In [81]:
sum(~dy_x)+sum(Xtrain['CAR_AV']!=1)

6456

In [82]:
len(dy_x2)

6456

In [66]:
label_vec = np.zeros((len(X),1))

In [67]:
label_vec

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 0.]])

In [ ]:
dy_x = model.predict(X_ml)

### Training

In [15]:
clf = LogisticRegression(multi_class='ovr').fit(Xtrain[ML_feat], ytrain)

In [16]:
clf2 = LogisticRegression(solver = "lbfgs", multi_class='multinomial').fit(Xtrain[ML_feat], ytrain)

In [17]:
clf_cv = LogisticRegressionCV(multi_class='ovr').fit(Xtrain[ML_feat], ytrain)
clf_cv2 = LogisticRegressionCV(multi_class='multinomial').fit(Xtrain[ML_feat], ytrain)

The Attribute "C\_" gives the value of the regularizer that is best for each class.  Because the 'refit' parameter is default true for this model, the whole model is refit on all of the training data after finding the best hyperparameter C for each class.

In [18]:
clf_cv.C_

array([  2.15443469e+01,   1.00000000e-04,   1.00000000e-04])

I think this is due to it being the multinomial case and learning the same parameters for the whole set.

In [19]:
clf_cv2.C_

array([ 0.35938137,  0.35938137,  0.35938137])

### Testing
We can predict on the whole set and then evaluate on the test and train separately.

In [20]:
py_x = clf.predict_proba(X[ML_feat])
dy_x = clf.predict(X[ML_feat])

In [27]:
ml_ut.print_predict(X[ML_feat], clf, ytrain, ytest, split)

Training Log Loss: 0.699283805597
Training Accuracy: 0.69165952585
Testing  Log Loss: 0.784816724333
Testing  Accuracy: 0.585687382298


The accuracy is substantially higher on the training set, which is somewhat surprising. Logistic Regression is not something that I think usually overfits.  Let's look at the other models.

In [29]:
ml_ut.print_predict(X[ML_feat], clf2, ytrain, ytest, split)

Training Log Loss: 0.815291533847
Training Accuracy: 0.651242502142
Testing  Log Loss: 0.962744442308
Testing  Accuracy: 0.478611783697


The accuracy for the multinomial fit was worse than the one-versus-all method for the non-cross-validated case.

#### With Cross-Validation

In [30]:
ml_ut.print_predict(X[ML_feat], clf_cv, ytrain, ytest, split)

Training Log Loss: 0.787363181432
Training Accuracy: 0.657812053699
Testing  Log Loss: 0.942837909585
Testing  Accuracy: 0.478880817864


In [32]:
ml_ut.print_predict(X[ML_feat], clf_cv2, ytrain, ytest, split)

Training Log Loss: 0.806953372192
Training Accuracy: 0.651242502142
Testing  Log Loss: 0.953894843485
Testing  Accuracy: 0.477804681195


Surprisingly, we do substantially better without the cross-validation.  Perhaps there is another setting that I am overlooking. I did see that the best C values were at the bounds of the search space, which could mean that the bounds need to be widened.

#### Increase bounds of C parameter
The best performing was the dead-simplest.  Let's try to increase the bounds on the C parameter.

In [33]:
np.logspace(-1,2,4)

array([   0.1,    1. ,   10. ,  100. ])

In [34]:
model = LogisticRegressionCV(Cs = np.logspace(-10,2,13)).fit(Xtrain[ML_feat], ytrain)

In [35]:
ml_ut.print_predict(X[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.802159414805
Training Accuracy: 0.659097400743
Testing  Log Loss: 1.00768252553
Testing  Accuracy: 0.474307237019


In [36]:
model.C_

array([  1.00000000e+01,   1.00000000e-06,   1.00000000e-10])

In [38]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)
ml_ut.print_predict(X[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.699283805597
Training Accuracy: 0.69165952585
Testing  Log Loss: 0.784816724333
Testing  Accuracy: 0.585687382298


In [39]:
model.C

1.0

### Issue of C parameter
The regularization parameter is the only real parameter to be tuned for logistic regression, besides possibly the issue of one-vs-all against multinomial.  It is possible that the first models learned with cross validation are **worse with less data**, and so they learn sub-optimal parameters.  That is the only explanation I can think of.  We can move on to the inclusion or exclusion of feature sets.

## Grouping within Cross-validation

There is a larger possible issue of structure within the data, meaning that the order of samples in the data set is actually informative to the result.  If there is such an order, the cross validation will be hurt dramatically.  We could keep the train-test split for the sake of testing robustness to this, but the cross-validation grouping does not need to have this.  The input and ouput vectors must be shuffled together.

In [48]:
Xtrain = copy.deepcopy(X[:split])

In [49]:
shuffle_index = np.random.permutation(Xtrain.index)

In [50]:
Xtrain = Xtrain.reindex(shuffle_index)
ytrain = ytrain.reindex(shuffle_index)

Now the groups should at least not be stuck when doing the cross-validation.

Quick check here to confirm that the training is working as well as before.

In [53]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)

In [52]:
def print_predict2(X_ml_train, X_ml_test, model, ytrain, ytest, split):
    py_x_tr = model.predict_proba(X_ml_train)
    dy_x_tr = model.predict(X_ml_train)
    py_x_te = model.predict_proba(X_ml_test)
    dy_x_te = model.predict(X_ml_test)
    print("Training Log Loss: " + str(sklearn.metrics.log_loss(ytrain, py_x_tr)))
    print("Training Accuracy: " + str(sklearn.metrics.accuracy_score(ytrain, dy_x_tr)))
    print("Testing  Log Loss: " + str(sklearn.metrics.log_loss(ytest, py_x_te)))
    print("Testing  Accuracy: " + str(sklearn.metrics.accuracy_score(ytest, dy_x_te)))
    

In [55]:
print_predict2(Xtrain[ML_feat], Xtest[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.699158354787
Training Accuracy: 0.691373893173
Testing  Log Loss: 0.784915488786
Testing  Accuracy: 0.584880279796


#### Now try with cross-validation

In [56]:
model = LogisticRegressionCV().fit(Xtrain[ML_feat], ytrain)

In [57]:
print_predict2(Xtrain[ML_feat], Xtest[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.722908215667
Training Accuracy: 0.676092544987
Testing  Log Loss: 0.822443493302
Testing  Accuracy: 0.540758676352


Still worse!

In [58]:
model.C_

array([  1.00000000e+04,   2.78255940e+00,   2.15443469e+01])

Perhaps multinomial would improve it

In [59]:
model = LogisticRegressionCV(multi_class='multinomial').fit(Xtrain[ML_feat], ytrain)

In [60]:
print_predict2(Xtrain[ML_feat], Xtest[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.803006862125
Training Accuracy: 0.652670665524
Testing  Log Loss: 0.949665233145
Testing  Accuracy: 0.476459510358


No better than before.

In [61]:
model.C_

array([ 21.5443469,  21.5443469,  21.5443469])

It does arrive at a parameter estimate 

In [34]:
model = LogisticRegressionCV(Cs = np.logspace(-10,2,13)).fit(Xtrain[ML_feat], ytrain)

### Cross-validation conclusion
For the purposes of this investigation into Logistic Regression, we can say that cross validation does not improve hyperparameter estimation because of the reduction in available data for each smaller training set.

# Feature Importances
To complete this examination, we should look at the weights learned for each input feature on the simplest case.

For the sake of clarity, we will repeat the data loading code here.

In [13]:
X = df_e.drop("CHOICE", axis = 1)
y = df_e['CHOICE']
split = 7002
Xtrain, Xtest, ytrain, ytest = ml_ut.tt_split(X,y,split)
ML_feat = pure_feats + simple_feats

In [14]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)

In [18]:
feat_weights = model.coef_

The target has 3 classes, so the coefficients in the one-versus-all case should be different for each possibility.  CHOICE 1,2,3 = Train, SM, Car

In [29]:
weights_TRAIN_choice = pd.DataFrame(list(zip(ML_feat, feat_weights[0])))
weights_TRAIN_choice.columns = ["feats", "weights"]
weights_TRAIN_choice["abs_val_weights"] = weights_TRAIN_choice["weights"].abs()

In [32]:
weights_TRAIN_choice.sort_values(by = "abs_val_weights", ascending=False)

,feats,weights,abs_val_weights
13,GA,1.647158,1.647158
9,SURVEY,-1.560206,1.560206
15,CAR_AV,-0.744071,0.744071
17,MALE,-0.346381,0.346381
12,AGE,0.345024,0.345024
16,FIRST,-0.144049,0.144049
6,SM_SEATS,0.078802,0.078802
14,SM_SEATS,0.078802,0.078802
10,LUGGAGE,-0.059824,0.059824
11,INCOME,0.032924,0.032924


#### Normalization is necessary
Should have thought of this before - the regularization parameter probably isn't very useful, even, when ranges of the input features are so disparate.  Examining the weights learned is not informative when there are some binary features and some pure numeric features.  Looking at the weights also shows that a broken-out model could be very useful.  The SURVEY and CAR_AV feats are very important because they encode where the survey happens and if a car is available at all.  

Next steps are to do the following:

Start with a normalization.

Break into two separate models:

Car v. no car

Of those predicted no car, plus those without car, predict TRAIN v. SM.  All these predictions aggregate and are evaluated against test set.  These are two logistic regression models.

Rerun the cross-validation for choosing the regularization parameter.

## Scratch

In [16]:
model.__dict__

{'C': 1.0,
 'class_weight': None,
 'classes_': array([1, 2, 3]),
 'coef_': array([[ -6.24196659e-03,  -1.56614335e-03,  -8.05115379e-03,
           7.24015906e-03,   9.84411207e-04,   1.21715247e-02,
           7.88019723e-02,   9.78567751e-04,  -2.32301360e-03,
          -1.56020575e+00,  -5.98237316e-02,   3.29235890e-02,
           3.45024209e-01,   1.64715805e+00,   7.88019723e-02,
          -7.44071117e-01,  -1.44049453e-01,  -3.46380749e-01],
        [  2.03824369e-03,   1.39944301e-03,   4.05240861e-03,
          -8.09453176e-03,  -9.66416735e-04,  -8.71912154e-03,
          -9.24285666e-02,   5.30383823e-03,   5.37788114e-03,
          -1.06404551e+00,  -1.58561803e-01,  -5.00998228e-02,
          -2.52697314e-01,  -1.04758297e+00,  -9.24285666e-02,
          -6.51870437e-01,   1.58652484e-01,   3.51000813e-01],
        [  4.44609225e-03,  -1.30528340e-04,  -2.19165814e-04,
           4.56986448e-03,   2.24049026e-04,  -3.57221475e-04,
           5.29144416e-02,  -1.17170580e-0